<a href="https://colab.research.google.com/github/2020-nlp-c/nlp-statisticsmodel/blob/master/bogyung/SpamFiltering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 스팸 필터링

In [1]:
import pandas as pd
mail = [["me free lottery", "spam"],
        ["free get free you", "spam"],
        ["you free scholarship", "normal"],
        ["free to contact me", "normal"],
        ["you won award", "normal"],
        ["you ticket lottery", "spam"]]
df = pd.DataFrame(mail, columns = ["tokens", "label"])
df

,tokens,label
0,me free lottery,spam
1,free get free you,spam
2,you free scholarship,normal
3,free to contact me,normal
4,you won award,normal
5,you ticket lottery,spam


In [2]:
df.tokens = df.tokens.map(lambda x: x.split())
df # input

,tokens,label
0,"[me, free, lottery]",spam
1,"[free, get, free, you]",spam
2,"[you, free, scholarship]",normal
3,"[free, to, contact, me]",normal
4,"[you, won, award]",normal
5,"[you, ticket, lottery]",spam


In [3]:
import numpy as np

tokens = []
for token in df.tokens:
    for i in range(len(token)):
        tokens.append(token[i])
tokens = np.unique(tokens)

spam = {}
normal = {}
for token in tokens:
    spam[token] = 0
    normal[token] = 0

In [4]:
print(spam)
print(normal)

{'award': 0, 'contact': 0, 'free': 0, 'get': 0, 'lottery': 0, 'me': 0, 'scholarship': 0, 'ticket': 0, 'to': 0, 'won': 0, 'you': 0}
{'award': 0, 'contact': 0, 'free': 0, 'get': 0, 'lottery': 0, 'me': 0, 'scholarship': 0, 'ticket': 0, 'to': 0, 'won': 0, 'you': 0}


In [5]:
for i, token_list in enumerate(df.tokens):
    for token in token_list:
        if df.label[i] == "spam":
            spam[token] += 1
        elif df.label[i] == "normal":
            normal[token] += 1

In [6]:
print(spam)
print(normal)

{'award': 0, 'contact': 0, 'free': 3, 'get': 1, 'lottery': 2, 'me': 1, 'scholarship': 0, 'ticket': 1, 'to': 0, 'won': 0, 'you': 2}
{'award': 1, 'contact': 1, 'free': 2, 'get': 0, 'lottery': 0, 'me': 1, 'scholarship': 1, 'ticket': 0, 'to': 1, 'won': 1, 'you': 2}


In [7]:
df_filter = pd.DataFrame({"spam":spam, "normal":normal})
df_filter

,spam,normal
award,0,1
contact,0,1
free,3,2
get,1,0
lottery,2,0
me,1,1
scholarship,0,1
ticket,1,0
to,0,1
won,0,1


In [8]:
k = 0.5
n_spam = np.sum(df_filter.spam)
n_normal = np.sum(df_filter.normal)

df_filter["P(w|spam)"] = (k+df_filter.spam)/(2*k+n_spam)
df_filter["P(w|normal)"] = (k+df_filter.normal)/(2*k+n_normal)
df_filter["Log(P(w|spam))"] = np.log(df_filter["P(w|spam)"])
df_filter["Log(P(w|normal))"] = np.log(df_filter["P(w|normal)"])

df_filter

,spam,normal,P(w|spam),P(w|normal),Log(P(w|spam)),Log(P(w|normal))
award,0,1,0.045455,0.136364,-3.091042,-1.992430
contact,0,1,0.045455,0.136364,-3.091042,-1.992430
free,3,2,0.318182,0.227273,-1.145132,-1.481605
get,1,0,0.136364,0.045455,-1.992430,-3.091042
lottery,2,0,0.227273,0.045455,-1.481605,-3.091042
me,1,1,0.136364,0.136364,-1.992430,-1.992430
scholarship,0,1,0.045455,0.136364,-3.091042,-1.992430
ticket,1,0,0.136364,0.045455,-1.992430,-3.091042
to,0,1,0.045455,0.136364,-3.091042,-1.992430
won,0,1,0.045455,0.136364,-3.091042,-1.992430


In [9]:
input_tokens = "free lottery".split()
P_spam = np.log( n_spam/(n_spam+n_normal) )
P_normal = np.log( n_normal/(n_spam+n_normal) )

for token in input_tokens:
    P_spam += float( df_filter[df_filter.index == token]["Log(P(w|spam))"] )
    P_normal += float( df_filter[df_filter.index == token]["Log(P(w|normal))"] )

P_spam_fin = np.exp(P_spam) / ( np.exp(P_spam) + np.exp(P_normal) )
P_normal_fin = np.exp(P_normal) / ( np.exp(P_spam) + np.exp(P_normal) )
print(round(P_spam_fin*100, 2),"%", round(P_normal_fin*100, 2),"%")

87.5 % 12.5 %


# SpamFiltering 클래스

In [10]:
import pandas as pd
import numpy as np

class SpamFiltering:
    def __init__(self, data, input_tokens, k = 0.5):
        self.data = data
        self.input_tokens = input_tokens
        self.k = k
    
    def percent_table(self):
        tokens = []
        for token in self.data.tokens:
            for i in range(len(token)):
                tokens.append(token[i])
        tokens = np.unique(tokens)
        
        spam = {}
        normal = {}
        for token in tokens:
            spam[token] = 0
            normal[token] = 0       

        for i, token_list in enumerate(self.data.tokens):
            for token in token_list:
                if self.data.label[i] == "spam":
                    spam[token] += 1
                elif self.data.label[i] == "normal":
                    normal[token] += 1

        df_filter = pd.DataFrame({"spam":spam, "normal":normal})
        n_spam = np.sum(df_filter.spam)
        n_normal = np.sum(df_filter.normal)
        df_filter["P(w|spam)"] = (self.k+df_filter.spam)/(2*self.k+n_spam)
        df_filter["P(w|normal)"] = (self.k+df_filter.normal)/(2*self.k+n_normal)
        df_filter["Log(P(w|spam))"] = np.log(df_filter["P(w|spam)"])
        df_filter["Log(P(w|normal))"] = np.log(df_filter["P(w|normal)"])
        return df_filter
    
    def p_spam(self):
        table = self.percent_table()
        P_spam = np.log( n_spam/(n_spam+n_normal) )
        P_normal = np.log( n_normal/(n_spam+n_normal) )
        
        for token in self.input_tokens.split():
            P_spam += float( table[table.index == token]["Log(P(w|spam))"] )
            P_normal += float( table[table.index == token]["Log(P(w|normal))"] )
        
        P_spam_fin = np.exp(P_spam) / ( np.exp(P_spam) + np.exp(P_normal) )
        return round(P_spam_fin, 2)
    
    def p_normal(self):
        return round(1-self.p_spam(), 2)
        
    def result(self):
        if self.p_spam() >= self.p_normal():
            return "spam"
        else:
            return "normal"

In [17]:
sf = SpamFiltering(data = df, input_tokens = "get lottery")
sf.percent_table()

,spam,normal,P(w|spam),P(w|normal),Log(P(w|spam)),Log(P(w|normal))
award,0,1,0.045455,0.136364,-3.091042,-1.992430
contact,0,1,0.045455,0.136364,-3.091042,-1.992430
free,3,2,0.318182,0.227273,-1.145132,-1.481605
get,1,0,0.136364,0.045455,-1.992430,-3.091042
lottery,2,0,0.227273,0.045455,-1.481605,-3.091042
me,1,1,0.136364,0.136364,-1.992430,-1.992430
scholarship,0,1,0.045455,0.136364,-3.091042,-1.992430
ticket,1,0,0.136364,0.045455,-1.992430,-3.091042
to,0,1,0.045455,0.136364,-3.091042,-1.992430
won,0,1,0.045455,0.136364,-3.091042,-1.992430


In [19]:
sf.p_spam(), sf.p_normal(), sf.result()

(0.94, 0.06, 'spam')

In [20]:
sf = SpamFiltering(data = df, input_tokens = "scholarship award")
sf.p_spam(), sf.p_normal(), sf.result()

(0.1, 0.9, 'normal')

## SpamFilteringFaster 클래스
- pandas 테이블 필요 없을 때

In [70]:
import pandas as pd
import numpy as np

class SpamFilteringFaster:
    def __init__(self, data, input_tokens, k = 0.5):
        self.data = data
        self.input_tokens = input_tokens
        self.k = k
    
    def percent(self):
        tokens = []
        for token in self.data.tokens:
            for i in range(len(token)):
                tokens.append(token[i])
        tokens = np.unique(tokens)
        
        spam = {}
        normal = {}
        for token in tokens:
            spam[token] = 0
            normal[token] = 0       

        for i, token_list in enumerate(self.data.tokens):
            for token in token_list:
                if self.data.label[i] == "spam":
                    spam[token] += 1
                elif self.data.label[i] == "normal":
                    normal[token] += 1

        n_spam = sum(spam.values())
        n_normal = sum(normal.values())

        log_spam = {}
        log_normal = {}

        for i, (s, n) in enumerate(zip(spam.values(), normal.values())):
            log_spam[tokens[i]] = np.log((k + s)/(2*k + n_spam))
            log_normal[tokens[i]] = np.log((k + n)/(2*k + n_normal))

        P_spam = np.log( n_spam/(n_spam+n_normal) )
        P_normal = np.log( n_normal/(n_spam+n_normal) )

        for token in self.input_tokens.split():
            P_spam += log_spam[token]
            P_normal += log_normal[token]

        P_spam_fin = np.exp(P_spam) / ( np.exp(P_spam) + np.exp(P_normal) )
        P_spam_fin = round(P_spam_fin, 2)
        return P_spam_fin, round(1 - P_spam_fin, 2)
    
    def result(self):
        if self.percent()[0] >= self.percent()[1]:
            return "spam"
        else:
            return "normal"

In [82]:
sff = SpamFilteringFaster(data = df, input_tokens = "get lottery")
sff.percent(), sff.result()

((0.94, 0.06), 'spam')

## 시간 비교

In [81]:
import time

# Class SpamFiltering
start = time.time()
sf = SpamFiltering(data = df, input_tokens = "get lottery")
sf.result()
print(time.time() - start)

# Class SpamFilteringFaster
start = time.time()
sff = SpamFilteringFaster(data = df, input_tokens = "get lottery")
sff.result()
print(time.time() - start)

0.02684497833251953
0.0017733573913574219
